# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import pickle
import sqlalchemy
import numpy

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

# number of parallel jobs in grid search (choose 4 for 8 GiB RAM, 16 for 32 GiB RAM)
n_jobs = 16

[nltk_data] Downloading package punkt to /home/zfsdt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/zfsdt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/zfsdt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/zfsdt/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('cleandata', engine)
# ignoring 'original' text because translation is available in message
# We can use 'genre' for model improvement later
X = df[['message', 'genre']]
# child_alone is always 0 in our data set
y = df.drop(columns=['id', 'message', 'original', 'genre', 'child_alone'])

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X['message'], y, test_size=.2, random_state=42)

In [6]:
pipeline.fit(X_train, y_train)

/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change 

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def report(y_test, y_pred):
    y_pred.columns = y_test.columns
    for column in y.columns:
        print('** {} **'.format(column).upper())
        print(classification_report(y_test[column], y_pred[column]))

y_pred = pd.DataFrame(pipeline.predict(X_test))
report(y_test, y_pred)

** RELATED **
              precision    recall  f1-score   support

           0       0.66      0.36      0.46      1245
           1       0.83      0.94      0.88      3998

   micro avg       0.80      0.80      0.80      5243
   macro avg       0.74      0.65      0.67      5243
weighted avg       0.78      0.80      0.78      5243

** REQUEST **
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      4352
           1       0.81      0.37      0.51       891

   micro avg       0.88      0.88      0.88      5243
   macro avg       0.85      0.68      0.72      5243
weighted avg       0.87      0.88      0.86      5243

** OFFER **
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00 

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# Below we will be using f1_macro for the grid search
f1_score(y_test, y_pred, average='macro')

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.20909426893826946

In [9]:
# Get initial parameters
# Below we will be using f1_macro for the grid search
pipeline.get_params()['clf']

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None)

In [10]:
parameters = {
    'clf__estimator__n_estimators' : (10, 20, 40),
    'clf__estimator__min_samples_split' : (2, 5, 10)
}   

# Vincent van Asch: Macro- and micro-averaged evaluation measures, 2013, Draft
# https://pdfs.semanticscholar.org/1d10/6a2730801b6210a67f7622e4d192bb309303.pdf
# suggests using a macro measure
cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_macro', verbose=10,
                  n_jobs=n_jobs, pre_dispatch=n_jobs)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 tasks      | elapsed:   35.9s
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:   55.1s
[Parallel(n_jobs=16)]: Done  15 out of  27 | elapsed:  1.1min remaining:   55.2s
[Parallel(n_jobs=16)]: Done  18 out of  27 | elapsed:  1.6min remaining:   49.2s
[Parallel(n_jobs=16)]: Done  21 out of  27 | elapsed:  1.7min remaining:   29.5s
[Parallel(n_jobs=16)]: Done  24 out of  27 | elapsed:  1.9min remaining:   14.5s
[Parallel(n_jobs=16)]: Done  27 out of  27 | elapsed:  2.2min remaining:    0.0s
[Parallel(n_jobs=16)]: Done  27 out of  27 | elapsed:  2.2min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=16,
       param_grid={'clf__estimator__n_estimators': (10, 20, 40), 'clf__estimator__min_samples_split': (2, 5, 10)},
       pre_dispatch=16, refit=True, return_train_score='warn',
       scoring='f1_macro', verbose=10)

In [11]:
cv.best_params_

{'clf__estimator__min_samples_split': 5, 'clf__estimator__n_estimators': 10}

In [12]:
cv.cv_results_

/usr/lib/python3/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/lib/python3/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/lib/python3/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/lib/python3/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are acc

{'mean_fit_time': array([26.48906477, 45.96980866, 82.56953756, 23.16636674, 39.35348129,
        66.4133447 , 19.83833178, 32.15436141, 50.77343146]),
 'std_fit_time': array([0.24620476, 0.46367228, 1.93708976, 0.25276014, 0.28186508,
        3.42886963, 0.24303901, 0.91196905, 0.46647022]),
 'mean_score_time': array([3.67842317, 4.99707143, 6.31753731, 3.74106105, 4.81713088,
        5.86042817, 3.5328962 , 4.1241912 , 5.44063942]),
 'std_score_time': array([0.15303503, 0.20848876, 0.29561906, 0.16939785, 0.09635885,
        0.44388662, 0.05940142, 0.25108387, 0.01724322]),
 'param_clf__estimator__min_samples_split': masked_array(data=[2, 2, 2, 5, 5, 5, 10, 10, 10],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_estimators': masked_array(data=[10, 20, 40, 10, 20, 40, 10, 20, 40],
              mask=[False, False, False, False, False, False, False, Fal

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
pipeline.set_params(**cv.best_params_)
pipeline.fit(X_train, y_train)
y_pred = pd.DataFrame(pipeline.predict(X_test))
f1_score(y_test, y_pred, average='macro')

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.25158948381342433

In [14]:
report(y_test, y_pred)

** RELATED **
              precision    recall  f1-score   support

           0       0.66      0.31      0.42      1245
           1       0.81      0.95      0.88      3998

   micro avg       0.80      0.80      0.80      5243
   macro avg       0.74      0.63      0.65      5243
weighted avg       0.78      0.80      0.77      5243

** REQUEST **
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4352
           1       0.82      0.45      0.58       891

   micro avg       0.89      0.89      0.89      5243
   macro avg       0.86      0.72      0.76      5243
weighted avg       0.88      0.89      0.88      5243

** OFFER **
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00 

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Try SVC instead of random forest

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X['message'], y, test_size=.2, random_state=42)
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC()))
])

pipeline2.fit(X_train, y_train)
y_pred = pipeline2.predict(X_test)
f1_score(y_test, y_pred, average='macro')

/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 

/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/lib/python3/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 

0.024722123456026715

In [16]:
parameters2 = [{'clf__estimator__kernel': ['rbf'],
                'clf__estimator__gamma': [1e-3, 1e-4],
                'clf__estimator__C': [1, 10, 100, 1000]},
               {'clf__estimator__kernel': ['linear'],
                'clf__estimator__C': [1, 10, 100, 1000]}]

cv2 = GridSearchCV(pipeline2, param_grid=parameters2, scoring='f1_macro', verbose=10, 
                   n_jobs=n_jobs, pre_dispatch=n_jobs)

cv2.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 tasks      | elapsed: 11.8min
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed: 13.5min
[Parallel(n_jobs=16)]: Done  16 tasks      | elapsed: 17.0min
/usr/lib/python3/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=16)]: Done  25 out of  36 | elapsed: 31.5min remaining: 13.9min
[Parallel(n_jobs=16)]: Done  29 out of  36 | elapsed: 36.5min remaining:  8.8min
[Parallel(n_jobs=16)]: Done  33 out of  36

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ity=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=16,
       param_grid=[{'clf__estimator__kernel': ['rbf'], 'clf__estimator__gamma': [0.001, 0.0001], 'clf__estimator__C': [1, 10, 100, 1000]}, {'clf__estimator__kernel': ['linear'], 'clf__estimator__C': [1, 10, 100, 1000]}],
       pre_dispatch=16, refit=True, return_train_score='warn',
       scoring='f1_macro', verbose=10)

In [17]:
#X_train, X_test, y_train, y_test = train_test_split(X['message'], y, test_size=.2, random_state=42)
pipeline2.set_params(**cv2.best_params_)
pipeline2.fit(X_train, y_train)
y_pred = pd.DataFrame(pipeline2.predict(X_test))
f1_score(y_test, y_pred, average='macro')

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.4200254801270426

In [18]:
cv2.best_params_

{'clf__estimator__C': 1000,
 'clf__estimator__gamma': 0.001,
 'clf__estimator__kernel': 'rbf'}

In [19]:
report(y_test, y_pred)

** RELATED **
              precision    recall  f1-score   support

           0       0.66      0.55      0.60      1245
           1       0.87      0.91      0.89      3998

   micro avg       0.83      0.83      0.83      5243
   macro avg       0.76      0.73      0.74      5243
weighted avg       0.82      0.83      0.82      5243

** REQUEST **
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4352
           1       0.79      0.61      0.69       891

   micro avg       0.91      0.91      0.91      5243
   macro avg       0.86      0.79      0.82      5243
weighted avg       0.90      0.91      0.90      5243

** OFFER **
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00 

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Include column 'genre'

In [20]:
ct = ColumnTransformer(transformers = [
        ('msg', Pipeline(steps = [
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
            ]), 'message'),
        ('gnr', OneHotEncoder(), ['genre'] )
    ])

In [21]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=.2, random_state=42)

In [22]:
pipeline3 = Pipeline([
    ('ct', ct),
    ('clf', MultiOutputClassifier(SVC()))
])

In [23]:
pipeline3.set_params(**cv2.best_params_)
pipeline3.fit(X_train2, y_train2)

Pipeline(memory=None,
     steps=[('ct', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('msg', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>,...ity=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
           n_jobs=None))])

In [24]:
y_pred2 = pipeline3.predict(X_test2)
f1_score(y_test2, y_pred2, average='macro')

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.41837717926247253

In [25]:
y_pred2 = pd.DataFrame(y_pred2)
report(y_test2, y_pred2)

** RELATED **
              precision    recall  f1-score   support

           0       0.66      0.55      0.60      1245
           1       0.87      0.91      0.89      3998

   micro avg       0.83      0.83      0.83      5243
   macro avg       0.76      0.73      0.74      5243
weighted avg       0.82      0.83      0.82      5243

** REQUEST **
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4352
           1       0.78      0.61      0.69       891

   micro avg       0.91      0.91      0.91      5243
   macro avg       0.85      0.79      0.81      5243
weighted avg       0.90      0.91      0.90      5243

** OFFER **
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00 

/usr/lib/python3/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

see https://scikit-learn.org/stable/modules/model_persistence.html

In [26]:
outfile = open('../models/classifier.pkl','wb')
pickle.dump(pipeline3, outfile)
outfile.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.